### CNN에 대한 구조와 동작

CNN의 구조  
CNN은 입력층, 합성곱층(Convolutional Layer), 활성화 함수(Activation Function), 풀링층(Pooling Layer), 완전 연결층(Fully Connected Layer), 출력층(Output Layer)으로 구성된다.

1 입력층 (Input Layer)
이미지를 입력으로 받는 층이다. 입력 이미지의 크기와 채널 수(예: 흑백 이미지 1채널, 컬러 이미지 3채널)에 따라 텐서의 형상이 결정된다. 예를 들어, 28x28의 흑백 이미지는 1x28x28 형태로 입력된다.

2 합성곱층 (Convolutional Layer)
합성곱층은 필터(커널)를 사용하여 입력 이미지에서 에지, 코너, 텍스처 등의 특징을 추출한다. 필터는 학습 가능한 파라미터이며, 슬라이딩 방식으로 이미지 위를 이동하며 연산을 수행한다. 각 필터는 특정한 패턴을 감지하며, 그 결과는 특징 맵(feature map)으로 출력된다.

3 활성화 함수 (Activation Function)
합성곱 연산 후에는 비선형성을 부여하기 위해 활성화 함수가 적용된다. 대표적으로 ReLU(Rectified Linear Unit) 함수가 사용되며, 이는 입력값이 0 이하일 경우 0, 0 이상일 경우 그대로 출력한다. 이를 통해 모델이 복잡한 함수를 학습할 수 있게 된다.

4 풀링층 (Pooling Layer)
풀링층은 특징 맵의 크기를 줄이면서 중요한 정보만 남긴다. 일반적으로 Max Pooling이 사용되며, 특정 영역 내 최대값을 선택한다. 이 과정을 통해 연산량이 줄어들고, 과적합을 방지하는 데 도움이 된다.

5 완전 연결층 (Fully Connected Layer)
풀링된 데이터를 1차원 벡터로 변환한 후 완전 연결층에 전달한다. 이 층에서는 전체 뉴런이 서로 연결되어 있으며, 고차원의 특징을 조합해 분류를 수행한다.

6 출력층 (Output Layer)
최종적으로 softmax 함수를 통해 각 클래스에 대한 확률을 출력한다. 이 결과를 바탕으로 입력 이미지가 어떤 클래스에 속하는지를 예측하게 된다.

CNN의 작동 원리   
CNN은 다음과 같은 절차로 입력 데이터를 처리한다:

특징 추출: 합성곱 연산을 통해 입력 이미지로부터 국소적인 특징(에지, 질감 등)을 추출한다.

비선형 변환: ReLU 활성화 함수를 통해 비선형성을 부여한다.

공간 정보 축소: 풀링 연산으로 특징 맵을 압축하여 중요 정보만 보존한다.

특징 벡터화 및 분류: 추출된 특징을 1차원 벡터로 변환한 후 완전 연결층을 통해 최종 클래스를 예측한다.

이러한 과정은 반복적으로 이루어지며, 모델은 점차 복잡한 특징을 학습해 나간다. 예를 들어 초기 합성곱층에서는 단순한 선이나 모서리를 감지하고, 이후 층으로 갈수록 더 고차원적인 패턴(예: 눈, 얼굴 형태 등)을 인식할 수 있게 된다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# 1. 데이터 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) 
]) # normalization 0.1307, 0.3081은 MNIST 데이터셋의 평균과 표준편차
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset  = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
# 2. CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)     # 1채널 입력, 10개 필터, 5x5
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)  # 숫자 0~9

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))   # 28x28 -> 12x12
        x = F.relu(F.max_pool2d(self.conv2(x), 2))   # 12x12 -> 4x4
        x = x.view(-1, 320)                          # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# 3. 학습 루프
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 함수
def train(model, loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # Negative log likelihood
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Train Epoch {epoch} [{batch_idx * len(data)}/{len(loader.dataset)}] Loss: {loss.item():.4f}")

# 실행
for epoch in range(1, 4):  # 간단히 3 에폭만
    train(model, train_loader, optimizer, epoch)
